# Clue Riddle
____

One rainy evening, Mr. Body killed 5 people at Clue Mansion. The people killed were Mrs. White, Mr. Green, Professor Plum, Colonel Mustard, and Ms. Scarlet. The murders took place in the kitchen, billiards room, conservatory, hall, and library. No two people were murdered with the same weapon. The weapons were candlestick, rope, revolver, knife, and lead pipe. From the clues given, try to determine the room in which each person was killed and the weapon used.
1. The murder with the lead pipe was not done in the hall or the library.
2. Mr. Green was not murdered in the kitchen.
3. The rope was not the murder weapon used in the library.
4. Neither Mrs. White nor Ms. Scarlet were murdered with the candlestick, the revolver, or the lead pipe.
5. The person who was murdered in the billiards room had just finished having dinner with Ms. Scarlet, Mr. Green, the person done in with the candlestick, and the victim of the rope.
6. Neither Mr. Green nor Professor Plum were killed with the lead pipe, in the hall, or in the library.

In [1]:
# libraries
import pandas as pd
import numpy as np
from itertools import permutations
import time

In [2]:
class Solver(object):

    def run(self):
        print('starting...')
        start_time = time.time()

        self.solve()

        elapsed_time = time.time() - start_time
        print('elapsed_time: {:.3f} sec'.format(elapsed_time))

    def solve(self):
        """
        One rainy evening, Mr. Body killed 5 people at Clue Mansion. The people killed were Mrs. White, Mr. Green,
        Professor Plum, Colonel Mustard, and Ms. Scarlet. The murders took place in the kitchen, billiards room,
        conservatory, hall, and library. No two people were murdered with the same weapon. The weapons were
        candlestick, rope, revolver, knife, and lead pipe. From the clues given, try to determine the room in which
        each person was killed and the weapon used.
        1. The murder with the lead pipe was not done in the hall or the library.
        2. Mr. Green was not murdered in the kitchen.
        3. The rope was not the murder weapon used in the library.
        4. Neither Mrs. White nor Ms. Scarlet were murdered with the candlestick, the revolver, or the lead pipe.
        5. The person who was murdered in the billiards room had just finished having dinner with Ms. Scarlet,
                Mr. Green, the person done in with the candlestick, and the victim of the rope.
        6. Neither Mr. Green nor Professor Plum were killed with the lead pipe, in the hall, or in the library.
        """

        people = ('white', 'green', 'plum', 'mustard', 'scarlet')
        places_start = ['kitchen', 'billiards', 'conservatory', 'hall', 'library']
        things_start = ['candlestick', 'rope', 'revolver', 'knife', 'pipe']

        matches = []

        white_idx = people.index('white')
        green_idx = people.index('green')
        plum_idx = people.index('plum')
        scarlet_idx = people.index('scarlet')

        for places in permutations(places_start):
            kitchen_idx = places.index('kitchen')
            billiards_idx = places.index('billiards')
            hall_idx = places.index('hall')
            library_idx = places.index('library')

            for things in permutations(things_start):
                candlestick_idx = things.index('candlestick')
                rope_idx = things.index('rope')
                revolver_idx = things.index('revolver')
                pipe_idx = things.index('pipe')

                # rule 1
                if pipe_idx == hall_idx or pipe_idx == library_idx:
                    continue

                # rule 2
                if green_idx == kitchen_idx:
                    continue

                # rule 3
                if rope_idx == library_idx:
                    continue

                # rule 4
                blacklist = {candlestick_idx, revolver_idx, pipe_idx}
                if white_idx in blacklist or scarlet_idx in blacklist:
                    continue

                # rule 5
                dinner_guests = {billiards_idx, scarlet_idx, green_idx, candlestick_idx, rope_idx}
                if len(dinner_guests) != 5:
                    continue

                # rule 6
                blacklist = {pipe_idx, hall_idx, library_idx}
                if green_idx in blacklist or plum_idx in blacklist:
                    continue

                # found a match
                print(people)
                print(places)
                print(things)
                matches.append((people, places, things))

        num_matches = len(matches)
        print('num_matches: {}'.format(num_matches))
        assert num_matches == 1


if __name__ == '__main__':
    instance = Solver()
    instance.run()

starting...
('white', 'green', 'plum', 'mustard', 'scarlet')
('hall', 'conservatory', 'kitchen', 'billiards', 'library')
('rope', 'revolver', 'candlestick', 'pipe', 'knife')
num_matches: 1
elapsed_time: 0.025 sec


## Scratch Code

In [ ]:
# helper function
def make_scenarios(people, places, things):
    """return a data frame with a row for each possible murder scenario
    """
    scenarios = []
    for person in people:
        for place in places:
            for thing in things:
                scenarios.append([person,place,thing])
                
    data = pd.DataFrame(scenarios,columns = ['Person','Place','Thing'])
    return data

In [ ]:
# generate data
people = ['white', 'green', 'plum', 'mustard', 'scarlet']
places = ['kitchen', 'billiards', 'conservatory', 'hall', 'library']
things = ['candlestick', 'rope', 'revolver', 'knife', 'pipe']

data = make_scenarios(people, places, things)
print(data.shape)
# data.sample(15)

In [ ]:
# conditions: (1 if scenario is possible, 0 otherwise)
data['cond_1'] = np.where((data['Thing'] == 'pipe') & (data['Place'].isin(['hall','library'])), 0, 1)
data['cond_2'] = np.where((data['Person'] == 'green') & (data['Place'] == 'kitchen'),0 ,1)
data['cond_3'] = np.where((data['Thing'] == 'rope') & (data['Place'] == 'library'), 0, 1)
data['cond_4'] = np.where(
    (data['Person'].isin(['white','scarlet'])) 
    & (data['Thing'].isin(['candlestick','revolver','pipe'])),
       0, 1)
data['cond_5'] = np.where(
    (data['Place'] == 'billiards')
    & ((data['Person'].isin(['green','scarlet'])) 
        |(data['Thing'].isin(['rope','candlestick'])) )
    ,0 ,1)
data['cond_6'] = np.where(
    (data['Person'].isin(['plum','green']))
    & (data['Thing'].isin(['pipe'])) | (data['Place'].isin(['hall','library'])),
    0, 1)

cond_list = ['cond_' + str(x) for x in range(1,7)]
data['valid'] = data[cond_list].sum(axis=1)

for col in cond_list:
    print(col, data.shape[0] - data[col].sum())

In [ ]:
# preview the valid scenarios only
# data.head(15)
solved = data[
    (data['valid'] == 6)
    
]
print(solved.shape)
solved

In [ ]:
solved.pivot_table(
    index = ['Person','Place'],
    values = ['valid'],
    aggfunc='count'
)